# Downloading data from kaggle

First you should create a token in kaggle and upload it in Files

In [1]:
! pip install -q kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c nlp-getting-started

mkdir: cannot create directory ‘/root/.kaggle’: File exists
  0% 0.00/593k [00:00<?, ?B/s]
100% 593k/593k [00:00<00:00, 82.8MB/s]


In [2]:
! unzip nlp-getting-started.zip

Archive:  nlp-getting-started.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


# Importing required libraries

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
plt.style.use('seaborn')

# Importing data

In [2]:
df = pd.read_csv("train.csv")
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [3]:
df.loc[df.text.isna(),:]

,id,keyword,location,text,target


First we create a simple model just by using the text and target column

In [4]:
data = df.iloc[:,[0,3,4]]

In [5]:
data.id = data.index.values + 1
data

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,id,text,target
0,1,Our Deeds are the Reason of this #earthquake M...,1
1,2,Forest fire near La Ronge Sask. Canada,1
2,3,All residents asked to 'shelter in place' are ...,1
3,4,"13,000 people receive #wildfires evacuation or...",1
4,5,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...
7608,7609,Two giant cranes holding a bridge collapse int...,1
7609,7610,@aria_ahrary @TheTawniest The out of control w...,1
7610,7611,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,7612,Police investigating after an e-bike collided ...,1


# Model

## Train/Test Split

In [6]:
from sklearn.model_selection import train_test_split
X = np.array(data.iloc[:,1])
y = np.array(data.iloc[:,-1])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=222)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=666)

##First layer

We use a pre-trained text embedding model as the first layer

In [7]:
import tensorflow_hub as hub
model = "https://tfhub.dev/google/nnlm-en-dim128-with-normalization/2"
hub_layer = hub.KerasLayer(model, input_shape=[], dtype=tf.string, trainable=True)
hub_layer(X_train[:3])

<tf.Tensor: shape=(3, 128), dtype=float32, numpy=
array([[ 4.65274483e-01,  6.03381395e-02, -2.25947276e-02,
         1.27515376e-01,  1.41367197e-01, -6.01468608e-02,
         5.61336055e-02, -8.17772746e-02, -1.25141844e-01,
         2.24831462e-01,  9.35670957e-02, -4.20897692e-01,
        -4.09677513e-02, -1.45227745e-01,  7.20147043e-02,
         1.95798203e-02, -4.88073304e-02,  2.30129459e-04,
        -6.65237084e-02,  7.47236833e-02, -1.82472244e-01,
         9.60581750e-02,  5.96922873e-05, -8.05068463e-02,
        -2.24518385e-02, -2.17670530e-01,  6.77260682e-02,
        -1.09929599e-01, -1.43464863e-01,  8.11961442e-02,
         8.42682868e-02,  1.66210532e-01,  7.72339106e-02,
         5.83183914e-02,  9.52747986e-02, -1.34607181e-01,
        -1.38968810e-01, -1.78631172e-01,  9.18664634e-02,
         2.02663898e-01, -9.43761542e-02,  1.13399848e-02,
        -4.26216424e-03, -3.19792554e-02,  5.60061038e-02,
         9.58899558e-02,  7.41870850e-02, -1.89713873e-02,
      

In [8]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 128)               124642688 
                                                                 
 dense (Dense)               (None, 16)                2064      
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 124,644,769
Trainable params: 124,644,769
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

In [10]:
history = model.fit(X_train,
                    y_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(X_val, y_val),
                    verbose=1)

Epoch 1/40
9/9 [==============================] - 2s 105ms/step - loss: 0.6730 - acc: 0.6024 - val_loss: 0.6440 - val_acc: 0.7111
Epoch 2/40
9/9 [==============================] - 1s 80ms/step - loss: 0.6073 - acc: 0.7677 - val_loss: 0.5986 - val_acc: 0.7505
Epoch 3/40
9/9 [==============================] - 1s 82ms/step - loss: 0.5459 - acc: 0.8045 - val_loss: 0.5539 - val_acc: 0.7728
Epoch 4/40
9/9 [==============================] - 1s 81ms/step - loss: 0.4799 - acc: 0.8408 - val_loss: 0.5144 - val_acc: 0.7899
Epoch 5/40
9/9 [==============================] - 1s 84ms/step - loss: 0.4115 - acc: 0.8647 - val_loss: 0.4850 - val_acc: 0.7984
Epoch 6/40
9/9 [==============================] - 1s 80ms/step - loss: 0.3484 - acc: 0.8899 - val_loss: 0.4651 - val_acc: 0.7978
Epoch 7/40
9/9 [==============================] - 1s 81ms/step - loss: 0.2931 - acc: 0.9109 - val_loss: 0.4556 - val_acc: 0.8024
Epoch 8/40
9/9 [==============================] - 1s 81ms/step - loss: 0.2388 - acc: 0.9323 - va

In [11]:
results = model.evaluate(X_test, y_test)

print(results)

48/48 [==============================] - 0s 7ms/step - loss: 0.9673 - acc: 0.7525
[0.9673160314559937, 0.7524622678756714]


In [12]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 128)               124642688 
                                                                 
 dense_2 (Dense)             (None, 16)                2064      
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 124,644,769
Trainable params: 124,644,769
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

In [14]:
history = model.fit(X,
                    y,
                    epochs=10,
                    batch_size=512,
                    verbose=1)

Epoch 1/10
15/15 [==============================] - 2s 80ms/step - loss: 0.5996 - acc: 0.8007
Epoch 2/10
15/15 [==============================] - 1s 75ms/step - loss: 0.4504 - acc: 0.8962
Epoch 3/10
15/15 [==============================] - 1s 75ms/step - loss: 0.3449 - acc: 0.9060
Epoch 4/10
15/15 [==============================] - 1s 75ms/step - loss: 0.2734 - acc: 0.9229
Epoch 5/10
15/15 [==============================] - 1s 76ms/step - loss: 0.2245 - acc: 0.9348
Epoch 6/10
15/15 [==============================] - 1s 76ms/step - loss: 0.1847 - acc: 0.9431
Epoch 7/10
15/15 [==============================] - 1s 75ms/step - loss: 0.1565 - acc: 0.9568
Epoch 8/10
15/15 [==============================] - 1s 75ms/step - loss: 0.1328 - acc: 0.9630
Epoch 9/10
15/15 [==============================] - 1s 76ms/step - loss: 0.1126 - acc: 0.9714
Epoch 10/10
15/15 [==============================] - 1s 76ms/step - loss: 0.0952 - acc: 0.9749


# Setting the test set

In [15]:
test = pd.read_csv('test.csv')
X_pred = np.array(test.iloc[:,-1])

In [18]:
y_pred = model.predict(X_pred)

In [20]:
y_pred[y_pred>=0.5] = 1
y_pred[y_pred<0.5] = 0

In [25]:
output = pd.DataFrame()
output['id'] = test['id']
output['target'] = y_pred
output['target'] = output['target'].astype(int)
output

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,0
3259,10865,1
3260,10868,1
3261,10874,1


In [26]:
output.to_csv('output.csv',index=False)

In [27]:
! kaggle competitions submit -c nlp-getting-started -f output.csv -m "first try"

100% 22.2k/22.2k [00:01<00:00, 16.1kB/s]
Successfully submitted to Natural Language Processing with Disaster Tweets